#RAG Routing e self-querying com AstraDB

In [ ]:
# Importar bibliotecas necessárias
import os
from astrapy import DataAPIClient
from astrapy.constants import VectorMetric
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import WebBaseLoader
from langchain.prompts import PromptTemplate
from langchain_core.output_parsers import JsonOutputParser
from langchain_core.output_parsers import StrOutputParser
import time

In [ ]:
# Instanciar o modelo de incorporação (neste exemplo, o Graphdoc)
from langchain_community.embeddings.fastembed import FastEmbedEmbeddings
embed_model = FastEmbedEmbeddings(model_name="graphdoc")

In [ ]:
# Instanciar o LLM (neste caso, usando Groq)
from groq import Groq
from langchain_groq import ChatGroq
from google.colab import userdata

llm = ChatGroq(
    temperatura=0,
    model_name="Llama3-8b-8192",
    api_key=userdata.get("GROQ_API_KEY"),
)

In [ ]:
# URLs dos documentos a serem carregados
urls = [
    "https://lilianweng.github.io/posts/2023-06-23-agent/",
    "https://lilianweng.github.io/posts/2023-03-15-prompt-engineering/",
    "https://lilianweng.github.io/posts/2023-10-25-adv-attack-llm/",
]

# Carregar documentos da web
docs = [WebBaseLoader(url).load() for url in urls]
docs_list = [item for sublist in docs for item in sublist]
print(f"len de documentos: {len(docs_list)}")

In [ ]:
# Dividir documentos em pedaços menores para se adequarem à janela de contexto do LLM
text_splitter = RecursiveCharacterTextSplitter.from_tiktoken_encoder(
    chunk_size=512, chunk_overlap=0
)
doc_splits = text_splitter.split_documents(docs_list)
print(f"Comprimento dos pedaços de documento gerados: {len(doc_splits)}")

In [ ]:
# Inicializar o cliente Astra
client = DataAPIClient(os.environ["ASTRA_DB_APPLICATION_TOKEN"])
database = client.get_database(os.environ["ASTRA_DB_API_ENDPOINT"])
print(f"* Database: {database.info().name}\n")

In [ ]:
# Criar uma coleção para armazenar os vetores
collection = database.create_collection(
    "vector_test",
    dimension=768,  # Certifique-se de ajustar para o tamanho de vetor da sua incorporação
    metric=VectorMetric.COSINE,  # Similaridade por cosseno
    check_exists=False,
)
print(f"* Collection: {collection.full_name}\n")

In [ ]:
# Carregar os documentos no Astra, associando os vetores de incorporação
for doc in doc_splits:
    embedding = embed_model.embed_query(doc.page_content)
    document = {
        "text": doc.page_content,
        "$vector": embedding,
    }
    collection.insert(document)

print(f"* Documentos inseridos na coleção: {len(doc_splits)}")

In [ ]:
# Realizar consulta de similaridade
query = "llm agent memory"
query_embedding = embed_model.embed_query(query)

results = collection.find(
    sort={"$vector": query_embedding},
    limit=5,  # Defina o limite de resultados desejado
    include_similarity=True,
)

print("Resultados da pesquisa vetorial:")
for doc in results:
    print("    ", doc["text"], " - Similaridade: ", doc["$similarity"])

In [ ]:
# Instanciar o roteador para redirecionar a consulta
prompt = PromptTemplate(
    template="""
    <|begin_of_text|><|start_header_id|>system<|end_header_id|> Você é um especialista em encaminhar uma 
    pergunta do usuário para um vectorstore ou pesquisa na web. Use o vectorstore para perguntas sobre agentes LLM, 
    engenharia de prompt e ataques adversários. Você não precisa ser rigoroso com as palavras-chave 
    na pergunta relacionadas a esses tópicos. Caso contrário, use a pesquisa na web. Dê uma escolha binária 'web_search' 
    ou 'vectorstore' com base na pergunta. Retorne um JSON com uma única chave 'datasource' e 
    nenhum preâmbulo ou explicação. Pergunta para encaminhar: {question} <|eot_id|><|start_header_id|>assistant<|end_header_id|>
    """,
    input_variables=["question"],
)

In [ ]:
# Roteamento da consulta baseado na pergunta
start = time.time()
question_router = prompt | llm | JsonOutputParser()

# Teste da cadeia de roteamento
question = "llm agent memory"
routing_decision = question_router.invoke({"question": question})
print(f"Decisão de roteamento: {routing_decision}")

end = time.time()
print(f"O tempo necessário para gerar resposta pela Router Chain em segundos: {end - start}")

# Resultado esperado: {'datasource': 'vectorstore'}